### Import

In [1]:
# import traci
import os, sys
#! default MacOS path of sumo tools
sys.path.append(os.path.join("/usr", "local", "opt", "sumo", "share", "sumo", "tools"))
# choose GUI or non GUI version
sumo_binary = "/usr/local/Cellar/sumo/1.15.0/bin/sumo"
#sumo_gui_binary = "/usr/local/Cellar/sumo/1.15.0/bin/sumo-gui"
#sumo_cmd = [sumo_binary, "-n", "/Users/jost/Workspace/traffic-simulation/xml/scenario1/grid.net.xml"]
sumo_cmd = [sumo_binary, "--start", "-c", "/Users/jost/Desktop/Magdeburgv2.sumocfg"]
import traci
from bs4 import BeautifulSoup
import numpy as np
import itertools
import warnings
import re

### Start SUMO / TRACI

In [2]:
# start traci
traci.start(sumo_cmd)

 Retrying in 1 seconds


***Starting server on port 57946 ***
Loading net-file from '/Users/jost/Desktop/../2022-10-20-17-02-51/osm.net.xml' ... done (2956ms).


Loading additional-files from '/Users/jost/Desktop/../2022-10-20-17-02-51/osm.poly.xml' ... done (4073ms).
Loading done.
Simulation version 1.15.0 started with time: 0.00.


(20, 'SUMO 1.15.0')

### Problems with traci
- `traci.lane.getFoes()` might use lane IDs twice for different connections
    - eg. junction at the FIN in Magdeburg

Solution: take net xml file, skip the ID matching and use foe matrix directly

In [3]:
with open("/Users/jost/2022-10-20-17-02-51/osm.net.xml", "r") as f:
    net_data = f.read()
net_xml = BeautifulSoup(net_data, "xml")

"""
for tls in traci.trafficlight.getIDList():
    get_eff_phases(tls)
"""

'\nfor tls in traci.trafficlight.getIDList():\n    get_eff_phases(tls)\n'

#### Method: `get_safe_phases(tls_id: str) -> total_safe_phases: list of lists`
- **INPUT:** tls ID as string
- **OUTPUT:** list of lists, where each inner lists contains connection indices that can share a green phase without colliding

**Concept:**  
Use the foe matrix of a nets xml file.  
A foe of a connection is another connection that might collide with it if they share the same green phase.  
  
Pairs of two connections sharing the same green phase without colliding (further named safe_phases or safe phase combinations) can be directly infered through the given foe matrix in a net xml file.  
Pairs of three, four, five and so on connections sharing the same green phase have to be computed based on the former safe phase combinations.  
  
The computation goes like:  
1. List pair of 2 connections using foe matrix
2. Intersect the safe connections of both connections in a pair of two to get the connections that are safe to combine with the pair
3. Combine all the intersected connections with the pair and do that for each pair to get pairs of 3 connections
4. For those pairs intersect the safe connections of all connections in the pair again to get further addable safe connections
5. Repeat the intersection and adding to combination steps until the intersections are empty i.e. there are no further combinable safe connections

In [12]:
# given a tls return all possible green combinations that are safe i.e. avoid collisions
#* INPUT: tls ID as string
#* OUTPUT: list of lists, where each inner lists means that given connection indices can share a green phase without colliding
#! make sure that indices are not changed during any parsing step
def get_safe_phases(tls_id: str):
    #TODO: modify param
    junction_id = tls_id 
    # whole junction including lanes etc
    junction_data = net_xml.find('junction', {'id': junction_id})
    
    #! throw warning if junction type is not traffic_light i.e. it is not controlled by tls
    junction_type = junction_data.attrs['type']
    if junction_type != 'traffic_light':
        warnings.warn(f"Junction with ID: {junction_id} is not of type 'traffic_light'. Instead it is of type: '{junction_type}'. This means that the junction is not controlled by a tls.")
        
    # just request data including foes, cont, index, response
    request_data = junction_data.find_all('request')
    # just foes
    foes = [request.attrs['foes'] for request in request_data]
    # foes as np array
    foes = np.ma.array([list(map(int, foe)) for foe in foes], mask=False)
    
    # number of tls / tls controlled connections
    n_tls = foes.shape[0]
    
    #! flip foes to be more intuitive, without flipping columns would be reversed i.e. last value in each row would correspond to index 0
    foes = np.fliplr(foes)
    
    # mask values corresponding to whether a connection has itself as foe or not
    # we want to ignore those for further computations
    for i in range(n_tls):
        foes.mask[i][i] = True
    
    # non-foes as a non sparse non binary list of foe indices
    # practically list of connection indices that don't collide i.e. that can share a green phase
    non_foes_ind = [np.where(foes[i] == 0)[0] for i in range(n_tls)]
    
    # placeholder to collect all safe_phases (safe phase combinations) during computation
    total_safe_phases = []
    
    # for each tls / tls controlled connection list all others that are no foes of each other
    # this combines all safe connection combinations which are directly readable from the foe matrix and which don't need further computation
    safe_phases = []
    for i in range(n_tls):
        # pairs of 2: extract from foes
        for i2 in non_foes_ind[i]:
            #* condition to not take duplicates e.g. [0, 1] and [1, 0]
            if i2 > i:
                safe_phases.append([i, i2])
            else:
                continue
    
    # add safe phases to collection
    total_safe_phases += safe_phases
    
    # placeholder for computation of next level non-foes
    curr_non_foes = []
    
    # for each safe_phase that was combined in the first step -> get the safe connections of the given safe_phase and intersect those safe connections with the possible safe connections of an connection added to the safe phase
    # the resulting safe phases contain more connections that can go green in parallel but also leads to less further possibilities to add to that combination
    for safe_phase in safe_phases:
        # start with first connection of a safe phase
        wip_non_foes = non_foes_ind[safe_phase[0]]
        # for each other connection of a safe phase get the step-wise intersection
        for connection in safe_phase[1:]:
            wip_non_foes = np.intersect1d(wip_non_foes, non_foes_ind[connection])
        curr_non_foes.append(wip_non_foes)
        
    # check if non foes exist
    # this is used as end condition for the loop
    non_foes_exist = False
    for non_foe in curr_non_foes:
        if non_foe.any():
            non_foes_exist = True
            break
    
    # iteratively look for safe_phases and further non_foes
    # stop when there are no further connection combinations having no_foe options to add
    while non_foes_exist: 
        # get new safe_phases
        #! duplicates allowed, filter befor appending
        dupl_safe_phases = []
        for i in range(len(safe_phases)):
            for connection in curr_non_foes[i]:
                dupl_safe_phases.append(safe_phases[i].copy() + [connection])
        # filter duplicates
        for safe_phase in dupl_safe_phases:
            safe_phase.sort()
        dupl_safe_phases.sort()
        # these are the newly computed safe phase combinations
        new_safe_phases = list(k for k,_ in itertools.groupby(dupl_safe_phases))
    
        # get new indices of non_foes for the newly computed safe phase combinations
        # so to speak: for each safe combinations a list of connection indices that could further be added to a given combination
        new_non_foes = []
        for i in range(len(new_safe_phases)):
            wip_non_foes = non_foes_ind[new_safe_phases[i][0]]
            for connection in new_safe_phases[i][1:]:
                wip_non_foes = np.intersect1d(wip_non_foes, non_foes_ind[connection])
            new_non_foes.append(wip_non_foes)
        
        # add safe phases to collection
        total_safe_phases += new_safe_phases
        
        # overwrite old values for next iteration
        curr_non_foes = new_non_foes.copy()
        # check if further non foes exist and set ending condition accordingly
        non_foes_exist = False
        for non_foe in curr_non_foes:
            if non_foe.any():
                non_foes_exist = True
                break
                
        safe_phases = new_safe_phases.copy()
    
    # return collection of all connection combinations that can share a green phase and would not lead to collisions
    # list of lists where each inner list contains indices of connections
    return total_safe_phases, n_tls

In [31]:
def safe_phases_to_bp_items(safe_phases, n_tls):
    print(safe_phases)
    all_connections = [i for i in range(n_tls)]
    print(all_connections)
    items = [i for i in range(len(safe_phases))]
    print(items)
    # TODO: later calc values from all connections in a safe phase based on traffic passthrough
    lengths = [len(safe_phase) for safe_phase in safe_phases]
    print(lengths)
    return items, all_connections, lengths
    

all_safe_phases, n_tls = get_safe_phases("cluster_244996795_33401116_4471941369_60603339")
bp_items, bp_required, bp_lengths = safe_phases_to_bp_items(all_safe_phases, n_tls)

[[0, 1], [0, 2], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [1, 2], [1, 6], [1, 7], [1, 8], [2, 3], [2, 4], [2, 5], [2, 6], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [3, 4], [3, 5], [3, 6], [3, 10], [3, 11], [3, 12], [3, 13], [4, 5], [4, 6], [4, 10], [4, 11], [4, 12], [4, 13], [5, 6], [6, 7], [6, 8], [6, 9], [6, 10], [6, 11], [7, 8], [7, 9], [7, 10], [7, 11], [8, 9], [8, 10], [8, 11], [9, 10], [9, 11], [10, 12], [10, 13], [11, 12], [11, 13], [12, 13], [0, 1, 2], [0, 1, 6], [0, 1, 7], [0, 1, 8], [0, 2, 5], [0, 2, 6], [0, 2, 9], [0, 2, 10], [0, 2, 11], [0, 2, 12], [0, 2, 13], [0, 5, 6], [0, 6, 7], [0, 6, 8], [0, 6, 9], [0, 6, 10], [0, 6, 11], [0, 7, 8], [0, 7, 9], [0, 7, 10], [0, 7, 11], [0, 8, 9], [0, 8, 10], [0, 8, 11], [0, 9, 10], [0, 9, 11], [0, 10, 12], [0, 10, 13], [0, 11, 12], [0, 11, 13], [0, 12, 13], [1, 2, 6], [1, 6, 7], [1, 6, 8], [1, 7, 8], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 10], [2, 3, 11], [2, 3, 12], [2, 3, 13], [2, 4, 5], [2, 4, 6],

In [28]:
# FIN intersection
all_safe_phases, n_tls = get_safe_phases("cluster_244996795_33401116_4471941369_60603339")

bp_items, bp_required, bp_lengths = safe_phases_to_bp_items(all_safe_phases, n_tls)

# Google OR Tools - The Bin Packing Problem tutorial
# https://developers.google.com/optimization/bin/bin_packing
from ortools.linear_solver import pywraplp

def create_data_model():
    """Create the data for the example."""
    data = {}
    # weights = capacity because only one safe phase should be picked per phase
    data['weights'] = [1] * len(bp_items)
    data['items'] = bp_items
    data['bins'] = data['items']
    data['bin_capacity'] = 1
    # added some data entries to use for solving
    data['values'] = bp_lengths
    data['required'] = bp_required
    data['safe_phases'] = all_safe_phases
    return data

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

data = create_data_model()

# Variables
# TODO: maybe change to: x[i, j] = len(i) if item i is packed in bin j?
    # to reinforce phases with many green connections
# TODO: even later change to x[i, j] = sum(values(i)) if item i is packed in bin j?
    # to reinforce phases with connections that have a high traffic passthrough
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

# tries to implement: 1 if conection con is in bin j
x2 = {}
for index, i in enumerate(data['items']):
    for con in data['safe_phases'][index]:
        for j in data['bins']:
            x2[(con, j)] = solver.IntVar(0, 1, 'x2_%i_%i' % (con, j))

# y[j] = 1 if bin j is used.
y = {}
for j in data['bins']:
    y[j] = solver.IntVar(0, 1, 'y[%i]' % j)
    
# Constraints
# TODO: change -> each item of an item must be in at least one bin
"""
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['bins']) == 1)
"""
# Each connection must be in at least one bin.
for con in data['required']:
    solver.Add(sum(x2[con, j] for j in data['bins']) >= 1)

# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
        sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
        data['bin_capacity'])
    
# Objective: minimize the number of bins used.
# TODO: could also be changed to maximize value of connections as well
solver.Minimize(solver.Sum([y[j] for j in data['bins']]))

status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    num_bins = 0
    for j in data['bins']:
        if y[j].solution_value() == 1:
            bin_items = []
            bin_weight = 0
            for i in data['items']:
                if x[i, j].solution_value() > 0:
                    bin_items.append(i)
                    bin_weight += data['weights'][i]
            if bin_weight > 0:
                num_bins += 1
                print('Bin number', j)
                print('  Items packed:', bin_items)
                print('  Total weight:', bin_weight)
                print()
    print()
    print('Number of bins used:', num_bins)
    print('Time = ', solver.WallTime(), ' milliseconds')
else:
    print('The problem does not have an optimal solution.')


Number of bins used: 0
Time =  2590  milliseconds


In [29]:
def get_unique_junction_ids(xml_net_path):
    with open(xml_net_path, "r") as f:
        net_data = f.read()
        
    # find all junctions controlled by a tls
    junction_list = re.findall(pattern="junction id=.* type=\"traffic_light\"", string=net_data)
    # extract junction ID from regex matches
    junction_list = [regex_match.split("\"")[1] for regex_match in junction_list]
    
    unique_junction_ids = []
    foe_matrices = []
    
    for junction_id in junction_list:
        junction_data = net_xml.find('junction', {'id': junction_id})
        request_data = junction_data.find_all('request')
        foes = [request.attrs['foes'] for request in request_data]
        foes_str = ""
        for foe in foes:
            foes_str += foe
        if foes_str in foe_matrices:
            continue
        else:
            foe_matrices.append(foes_str)
            unique_junction_ids.append(junction_id)
            
    return unique_junction_ids

In [30]:
unique_junction_list = get_unique_junction_ids("/Users/jost/2022-10-20-17-02-51/osm.net.xml")

In [28]:
for junction_id in unique_junction_list:
    get_safe_phases(junction_id)

In [10]:
def count_same_topology():
    
    with open("/Users/jost/2022-10-20-17-02-51/osm.net.xml", "r") as f:
        net_data = f.read()
    # find all junctions controlled by a tls
    junction_list = re.findall(pattern="junction id=.* type=\"traffic_light\"", string=net_data)
    # extract junction ID from regex matches
    junction_list = [regex_match.split("\"")[1] for regex_match in junction_list]
    print(f"There are {len(junction_list)} junctions controlled by a tls.")
    
    # TODO: for each junctionid get foes np array, put them into another np array and use np.unique() on that
    # collect all foe matrices including duplicates
    foe_matrices = []
    # also count rotated foe matrices
    rotated_foe_matrices = []
    
    net_xml = BeautifulSoup(net_data, "xml")
    
    # get foe matrix for each junction_id controlled by a tls
    for junction_id in junction_list:
        junction_data = net_xml.find('junction', {'id': junction_id})
        request_data = junction_data.find_all('request')
        foes = [request.attrs['foes'] for request in request_data]
        #print(foes)
        # sort foes to get rotated topologies
        rotated_foes = sorted(foes)
        #print(rotated_foes)
        foes_str = ""
        rotated_str = ""
        for foe in foes:
            foes_str += foe
        for foe in rotated_foes:
            rotated_str += foe
        foe_matrices.append(foes_str)
        rotated_foe_matrices.append(rotated_str)
    
    foe_matrices = set(foe_matrices)
    rotated_foe_matrices = set(rotated_foe_matrices)
    print(rotated_foe_matrices)
    print(f"There are {len(foe_matrices)} unique junction topologies controlled by a tls.")
    print(f"If rotation is allowed i.e. the index of a connection doesn't matter, there are {len(rotated_foe_matrices)} unique junction topologies controlled by a tls.")

In [11]:
count_same_topology()

There are 195 junctions controlled by a tls.
{'000000100001000000111110000001000010000011000110001000000100001100000110010000001000010000010000011011100000011101100000100000100000110000000111', '0000000000100000000001000000000000000110001111100000011111000110000010000100001000010000010000100010000010000100001000010000010000111110000001100100001000000111010000100000100001000010000100000110000001111100011000111110000001111100011000001110000001000010', '000100010001000110011110000111100010001000100010001001000010010010000100010010001000011000011110011011110000100100010000110110000011111000010001', '000001000010000001100011000001111100000010000100000100000100001000001000001000010000001111100000010000100000011000000111110001100000111000000110', '0000000000000000000000000', '0000010001000001111000000111100000100010000100001001100000000110000000011111000010000100001110000011', '0000100011000011110000010001000010000100010000100001000100000111100000100010000010001000001100000111', '000000110000

In [33]:
def get_efficient_phases(tls_id: str):
    # extract junction ID from TLS ID -> ignore "GS_" at the beginning
    #! this might be net-specific
    if tls_id.startswith("GS_"):
        junction_id = tls_id[3:]
    else:
        junction_id = tls_id
    
    # whole junction including lanes etc
    junction_data = net_xml.find('junction', {'id': junction_id})
    
    #! throw warning if junction type is not traffic_light i.e. it is not controlled by tls
    junction_type = junction_data.attrs['type']
    if junction_type != 'traffic_light':
        warnings.warn(f"Junction with ID: {junction_id} is not of type 'traffic_light'. Instead it is of type: '{junction_type}'. This means that the junction is not controlled by a tls.")
        
    # just request data including foes, cont, index, response
    request_data = junction_data.find_all('request')
    n_tls = [i for i in range(len(request_data))]
    print(n_tls)
    
    # get all safe phases for same tls_id
    all_safe_phases = get_safe_phases(tls_id)
    # reverse list of safe phases to get more combined connections first
    all_safe_phases.reverse()
    
    # always pick first
    # ! this yields the assumption that the first combination is never a bad choice and simplifies the algorithm
    efficient_phases = [all_safe_phases[0]]
    for connection in all_safe_phases[0]:
        n_tls.remove(connection)
    print(n_tls)
    
    
    print(all_safe_phases)
    for connections in all_safe_phases[1:]:
        if not n_tls:
            break
        else:
            for connection in connections:
                if connection in n_tls:
                    n_tls.remove(connection)
                    efficient_phases.append(connections)
                if not n_tls:
                        break
    #efficient_phases = set(efficient_phases)
        
    print(efficient_phases)
    
    
    # TODO: pick first entries containing connections and check all other occuring connections
    # TODO: repeat until each connection in n_tls is checked
    

In [34]:
get_efficient_phases("GS_cluster_244996795_33401116_4471941369_60603339")

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0, 1, 5, 6, 7, 8, 9, 10]
[[2, 3, 4, 11, 12, 13], [2, 3, 4, 10, 12, 13], [0, 6, 7, 8, 9, 11], [0, 6, 7, 8, 9, 10], [6, 7, 8, 9, 11], [6, 7, 8, 9, 10], [3, 4, 11, 12, 13], [3, 4, 10, 12, 13], [2, 4, 11, 12, 13], [2, 4, 10, 12, 13], [2, 3, 11, 12, 13], [2, 3, 10, 12, 13], [2, 3, 4, 12, 13], [2, 3, 4, 11, 13], [2, 3, 4, 11, 12], [2, 3, 4, 10, 13], [2, 3, 4, 10, 12], [2, 3, 4, 6, 11], [2, 3, 4, 6, 10], [2, 3, 4, 5, 6], [0, 7, 8, 9, 11], [0, 7, 8, 9, 10], [0, 6, 8, 9, 11], [0, 6, 8, 9, 10], [0, 6, 7, 9, 11], [0, 6, 7, 9, 10], [0, 6, 7, 8, 11], [0, 6, 7, 8, 10], [0, 6, 7, 8, 9], [0, 2, 11, 12, 13], [0, 2, 10, 12, 13], [0, 2, 6, 9, 11], [0, 2, 6, 9, 10], [0, 1, 6, 7, 8], [7, 8, 9, 11], [7, 8, 9, 10], [6, 8, 9, 11], [6, 8, 9, 10], [6, 7, 9, 11], [6, 7, 9, 10], [6, 7, 8, 11], [6, 7, 8, 10], [6, 7, 8, 9], [4, 11, 12, 13], [4, 10, 12, 13], [3, 11, 12, 13], [3, 10, 12, 13], [3, 4, 12, 13], [3, 4, 11, 13], [3, 4, 11, 12], [3, 4, 10, 13], [3, 4, 10, 12]

### Close SUMO / TRACI

In [8]:
# end
traci.close()